In [59]:
# Main imports
from econml.panel.dml import DynamicDML
from econml.dml import LinearDML
from tests_copy import DynamicPanelDGP, add_vlines

# Helper imports
import numpy as np
from sklearn.linear_model import Lasso, LassoCV, LogisticRegression, LogisticRegressionCV, MultiTaskLassoCV
import matplotlib.pyplot as plt

import pandas as pd

%matplotlib inline

In [3]:
# Define DGP parameters
np.random.seed(123)
n_panels = 5000 # number of panels
n_periods = 3 # number of time periods in each panel
n_treatments = 2 # number of treatments in each period
n_x = 100 # number of features + controls
s_x = 10 # number of controls (endogeneous variables)
s_t = 10 # treatment support size

In [4]:
# Generate data
dgp = DynamicPanelDGP(n_periods, n_treatments, n_x).create_instance(
            s_x, random_seed=12345)
Y, T, X, W, groups = dgp.observational_data(n_panels, s_t=s_t, random_seed=12345)
true_effect = dgp.true_effect

In [26]:
W_df = pd.DataFrame(W, columns=[f"W_{i}" for i in range(W.shape[1])])
T_df = pd.DataFrame(T, columns=[f"T_{i}" for i in range(T.shape[1])])

In [34]:
df = pd.concat([W_df, T_df, pd.Series(Y, name="Y"), pd.Series(groups, name="Groups")], axis=1)

In [41]:
true_effect

array([[0.05509021, 0.05760207],
       [0.29118846, 0.24326831],
       [0.13030029, 0.60557796]])

# Dynamic

In [50]:
est = DynamicDML(
    model_y=LassoCV(cv=3, max_iter=1000), 
    model_t=MultiTaskLassoCV(cv=3, max_iter=1000), 
    #fit_cate_intercept=False,
    cv=3)

est.fit(X=None, W=W, Y=Y, T=T, groups=groups)

In [52]:
est.intercept_

array([0.04000235, 0.0701606 , 0.31611764, 0.23714736, 0.13108411,
       0.60656886])

In [79]:
est.const_marginal_effect()

array([[0.04000235, 0.0701606 , 0.31611764, 0.23714736, 0.13108411,
        0.60656886]])

# Bootstrap

In [57]:
df_sample = df.groupby("Groups").apply(lambda x: x.sample(1))

In [60]:
W_fts = [col for col in df_sample.columns if "W" in col]
T_fts = [col for col in df_sample.columns if "T" in col]

In [73]:
est_l = LinearDML(
    model_y=LassoCV(cv=3, max_iter=1000), 
    model_t=MultiTaskLassoCV(cv=3, max_iter=1000), 
    #fit_cate_intercept=False,
    cv=3)

est_l.fit(X=None, W=df_sample[W_fts], Y=df_sample["Y"], T=df_sample[T_fts])

est_l.intercept_

array([0.13365554, 0.60411655])

In [74]:
est_l = LinearDML(
    model_y=LassoCV(cv=3, max_iter=1000), 
    model_t=MultiTaskLassoCV(cv=3, max_iter=1000), 
    #fit_cate_intercept=False,
    cv=3)

est_l.fit(X=None, W=W, Y=Y, T=T)

est_l.intercept_

array([0.13131142, 0.60666462])

In [75]:
est_l.const_marginal_ate()

array([0.13131142, 0.60666462])